![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [36]:
%pip install quantstats

Note: you may need to restart the kernel to use updated packages.


In [37]:
import json
import pandas as pd
import quantstats as qs
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# --- 1. Load the Results File ---
file_path = 'backtests/2025-09-11_11-24-18/1147155884.json'

with open(file_path, 'r') as f:
    results = json.load(f)

print("✅ Backtest results loaded successfully!")


# --- 2. Process the Data ---
equity_points = results['charts']['Strategy Equity']['series']['Equity']['values']
equity_df = pd.DataFrame(equity_points)
equity_df.rename(columns={0: 'time', 1: 'equity'}, inplace=True)
equity_df['time'] = pd.to_datetime(equity_df['time'], unit='s')

# --- THE FIX IS HERE ---
# We must set the 'time' column as the index before using time-series functions.
equity_df = equity_df.set_index('time')

# Calculate daily returns for the bar chart
equity_df['returns'] = equity_df['equity'].pct_change()

# Resample to daily OHLC for the candlestick chart
equity_ohlc = equity_df['equity'].resample('D').ohlc().dropna()


# --- 3. Create the Combined Chart ---
print("\nGenerating advanced equity chart...")

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                    vertical_spacing=0.03, row_heights=[0.7, 0.3])

fig.add_trace(go.Candlestick(x=equity_ohlc.index,
                             open=equity_ohlc['open'],
                             high=equity_ohlc['high'],
                             low=equity_ohlc['low'],
                             close=equity_ohlc['close'],
                             name='Equity'),
              row=1, col=1)

fig.add_trace(go.Bar(x=equity_df.index, y=equity_df['returns'], name='Daily Return'),
              row=2, col=1)

# --- 4. Style the Chart ---
fig.update_layout(
    title='Strategy Equity',
    yaxis1_title='Equity ($)',
    yaxis2_title='Return (%)',
    xaxis2_title='Date',
    xaxis1_rangeslider_visible=False,
    xaxis2_rangeslider_visible=False,
    showlegend=False,
    height=600
)
fig.update_yaxes(tickformat=".2%", row=2, col=1)

fig.show()


# --- 5. Generate the Full Statistics Report ---
qs.reports.html(equity_df['equity'], output='strategy_report.html', title='VolatilityShield Performance')
print("✅ Report saved! Open 'strategy_report.html' in your browser.")

✅ Backtest results loaded successfully!

Generating advanced equity chart...


✅ Report saved! Open 'strategy_report.html' in your browser.


In [38]:
import pandas as pd
import quantstats as qs
import re

# This assumes the 'results' and 'equity_df' variables are already loaded from the cell above

def parse_stat_to_float(stat_value):
    """
    Safely parses a value from the statistics dictionary into a float.
    """
    if isinstance(stat_value, (int, float)):
        return float(stat_value)
    if isinstance(stat_value, str):
        cleaned_str = re.sub(r'[$,()\s]', '', stat_value)
        try:
            return float(cleaned_str)
        except (ValueError, TypeError):
            return 0.0
    return 0.0

# --- Recreate the QuantConnect-style Summary Bar (Accurate Version) ---

starting_cash = 100000.0
stats = results.get('statistics', {})
orders = results.get('orders', {})

final_equity = results['charts']['Strategy Equity']['series']['Equity']['values'][-1][1]
net_profit_val = final_equity - starting_cash
return_pct_val = net_profit_val / starting_cash

total_volume = 0
for order_id, order_data in orders.items():
    order_value = order_data.get('value', abs(order_data.get('quantity', 0) * order_data.get('price', 0)))
    total_volume += abs(order_value)

daily_returns = equity_df.pct_change().dropna()
sharpe_ratio_series = qs.stats.sharpe(daily_returns)
total_fees_val = parse_stat_to_float(stats.get('Total Fees', 0))

# --- THE FIX IS HERE ---
# Instead of .item(), we access the first element of the Series by its position [0]
sharpe_ratio_val = sharpe_ratio_series.iloc[0]

summary_data = {
    'Equity': f"${final_equity:,.2f}",
    'Fees': f"(${abs(total_fees_val):,.2f})",
    'Net Profit': f"${net_profit_val:,.2f}",
    'Sharpe Ratio': f"{sharpe_ratio_val:.3f}",
    'Return %': f"{return_pct_val:.3%}",
    'Volume': f"${total_volume:,.2f}"
}

summary_df = pd.DataFrame([summary_data])

print("--- Backtest Summary ---")
display(summary_df)

--- Backtest Summary ---


,Equity,Fees,Net Profit,Sharpe Ratio,Return %,Volume
0,"$132,307.29",($4.38),"$32,307.29",0.266,32.307%,"$580,416.12"
